<a href="https://colab.research.google.com/github/febrahim-driod/hello-world/blob/master/notebook/Yolov8_Car_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt) and check software and hardware.

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
!pip install roboflow --quiet
!pip install clearml --quiet

Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.4/78.2 GB disk)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 972.3/972.3 KB 15.6 MB/s eta 0:00:00


In [7]:
import os
from IPython.display import Image, display
import glob
from clearml import Task

HOME = os.getcwd()
print(HOME)

/content


In [3]:
model_type='detection'
model_type='body_parts'
#model_type='severity'
#model_type='damage_type'

WEIGHTS_PATH = f"{HOME}/yolov8l-seg.pt"
TASK='segment'
IMGSZ=640
BATCH_SIZE=16
WORKERS=8
PATIENCE=5
WEIGHTS_PATH = f"{HOME}/yolov8x-seg.pt"


if model_type=="detection":
   EPOCH=30
   PROJECT_NAME="yolov8_damage_detection"
   RUN_NAME="train_seg_{"+str(EPOCH)+"}_epoch"
elif model_type=="body_parts":
   EPOCH=50
   PROJECT_NAME="yolov8_body_parts"
   RUN_NAME="train_seg_{"+str(EPOCH)+"}_epoch"
elif model_type=="severity":
   EPOCH=50
   PROJECT_NAME="yolov8_severity"
   RUN_NAME="train_seg_{"+str(EPOCH)+"}_epoch"
elif model_type=="damage_type":
   EPOCH=50
   PROJECT_NAME="yolov8_damage_type"
   RUN_NAME="train_seg_{"+str(EPOCH)+"}_epoch"
   

In [6]:


from roboflow import Roboflow
rf = Roboflow(api_key="P5E6JhfkeiA1nAdZAWnB")
if model_type=="detection":
  project = rf.workspace("sinfo").project("car_damage-4xqh8")
  dataset = project.version(6).download("yolov8")
  DATA_YAML_PATH= f"{HOME}/car_damage-6/data.yaml"
elif model_type=="body_parts":
  project = rf.workspace("sinfo").project("car-body-parts-p025a")
  dataset = project.version(5).download("yolov8")
  DATA_YAML_PATH= f"{HOME}/car-body-parts-5/data.yaml"
elif model_type=="severity":
  project = rf.workspace("sinfo").project("car-body-parts-p025a")
  dataset = project.version(5).download("yolov8")
  DATA_YAML_PATH= f"{HOME}/car-damage-severity-2/data.yaml"
elif model_type=="damage_type":
  project = rf.workspace("sinfo").project("car-body-parts-p025a")
  dataset = project.version(5).download("yolov8")
  DATA_YAML_PATH= f"{HOME}/car-damage-type-1/data.yaml"


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.55, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to car-body-parts-5 in yolov8:: 100%|██████████| 920/920 [00:00<00:00, 1219.14it/s]


In [4]:


%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# colob_notebook
%env CLEARML_API_ACCESS_KEY=AMY0O903Z5A7KN5FETFJ
%env CLEARML_API_SECRET_KEY=y7hUwY62xMFWwADGFWzKd33C2b8C4f1AxTDymedak5lQFzmbJg

task = Task.init(project_name=PROJECT_NAME, task_name=RUN_NAME)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 972.3/972.3 KB 13.3 MB/s eta 0:00:00
env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=AMY0O903Z5A7KN5FETFJ
env: CLEARML_API_SECRET_KEY=y7hUwY62xMFWwADGFWzKd33C2b8C4f1AxTDymedak5lQFzmbJg


# 1. Predict

YOLOv8 may be used directly in the Command Line Interface (CLI) with a `yolo` command for a variety of tasks and modes and accepts additional arguments, i.e. `imgsz=640`. See a full list of available `yolo` [arguments](https://docs.ultralytics.com/usage/cfg/) in the YOLOv8 [Docs](https://docs.ultralytics.com).


In [6]:
# Run inference on an image with YOLOv8n-seg
!yolo predict model=yolov8l-seg.pt source='https://ultralytics.com/images/zidane.jpg' box=False

100% 88.1M/88.1M [00:01<00:00, 81.0MB/s]
Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-seg summary (fused): 295 layers, 45973568 parameters, 0 gradients, 220.5 GFLOPs

100% 165k/165k [00:00<00:00, 7.32MB/s]
image 1/1 /content/zidane.jpg: 384x640 2 persons, 2 ties, 50.7ms
Speed: 0.7ms preprocess, 50.7ms inference, 70.7ms postprocess per image at shape (1, 3, 640, 640)


# 3. Train

<p align=""><a href="https://roboflow.com/?ref=ultralytics"><img width="1000" src="https://github.com/ultralytics/assets/raw/main/yolov8/banner-integrations.png"/></a></p>

Train YOLOv8 on [Detection](https://docs.ultralytics.com/tasks/detect/), [Segmentation](https://docs.ultralytics.com/tasks/segment/) and [Classification](https://docs.ultralytics.com/tasks/classify/) datasets.

'/content/car_damage-6/data.yaml'

In [9]:
# Train YOLOv8m on Custom Data
!yolo mode=train \
      model=$WEIGHTS_PATH \
      data=$DATA_YAML_PATH \
      epochs=$EPOCH \
      imgsz=$IMGSZ \
      task=$TASK \
      project=$PROJECT_NAME \
      name=$RUN_NAME \
      batch=$BATCH_SIZE \
      workers=$WORKERS

Ultralytics YOLOv8.0.55 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=/content/yolov8l-seg.pt, data=/content/car_damage-6/data.yaml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolov8_damage_detection, name=train_seg_20_epoch, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False

In [ ]:
from IPython.display import Image, display
import glob

for imageName in glob.glob('/content/PROJECT_NAME/RUN_NAME/train_*.jpg'):
      display(Image(filename=imageName))
      print("\n")



In [ ]:


for imageName in glob.glob('/content/PROJECT_NAME/RUN_NAME/val_*.jpg'):
      display(Image(filename=imageName))
      print("\n")

# 5. Predict
Predict a model's accuracy on the dataset's `test` splits. 

In [ ]:
!yolo task=$TASK \
      mode=predict \
      model='/content/PROJECT_NAME/RUN_NAME/weights/best.pt' \
      source='/content/car-damage-segmentation-2/test/images' \
      show=False \
      imgsz=$IMGSZ \
      project=$PROJECT_NAME \
      name=$RUN_NAME \
      hide_labels=True \
      conf=0.25 \
      save=True \
      box=False

In [ ]:
from IPython.display import Image, display
import glob

for imageName in glob.glob('/content/yolov8_damage_detection/train2/*.jpg')[90:100]:
      display(Image(filename=imageName))
      print("\n")

In [ ]:
#Close clearML experiment
task.close()

# 4. Export

Export a YOLOv8 model to any supported format with the `format` argument, i.e. `format=onnx`.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                                     | `format=`          | Model                     |
|----------------------------------------------------------------------------|--------------------|---------------------------|
| [PyTorch](https://pytorch.org/)                                            | -                  | `yolov8n.pt`              |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)                    | `torchscript`      | `yolov8n.torchscript`     |
| [ONNX](https://onnx.ai/)                                                   | `onnx`             | `yolov8n.onnx`            |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)                     | `openvino`         | `yolov8n_openvino_model/` |
| [TensorRT](https://developer.nvidia.com/tensorrt)                          | `engine`           | `yolov8n.engine`          |
| [CoreML](https://github.com/apple/coremltools)                             | `coreml`           | `yolov8n.mlmodel`         |
| [TensorFlow SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`      | `yolov8n_saved_model/`    |
| [TensorFlow GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`               | `yolov8n.pb`              |
| [TensorFlow Lite](https://www.tensorflow.org/lite)                         | `tflite`           | `yolov8n.tflite`          |
| [TensorFlow Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`          | `yolov8n_edgetpu.tflite`  |
| [TensorFlow.js](https://www.tensorflow.org/js)                             | `tfjs`             | `yolov8n_web_model/`      |
| [PaddlePaddle](https://github.com/PaddlePaddle)                            | `paddle`           | `yolov8n_paddle_model/`   |



In [ ]:
!yolo export model='/content/PROJECT_NAME/RUN_NAME/weights/best.pt' format=onnx

# 5. Python Usage

YOLOv8 was reimagined using Python-first principles for the most seamless Python YOLO experience yet. YOLOv8 models can be loaded from a trained checkpoint or created from scratch. Then methods are used to train, val, predict, and export the model. See a detailed Python usage examples in the YOLOv8 [Docs](https://docs.ultralytics.com/usage/python/).

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data='coco128.yaml', epochs=3)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
success = model.export(format='onnx')  # export the model to ONNX format

# 6. Tasks

YOLOv8 can train, val, predict and export models for the 3 primary tasks in vision AI: detection, segmentation and classification.

<img width="1024" src="https://user-images.githubusercontent.com/26833433/212094133-6bb8c21c-3d47-41df-a512-81c5931054ae.png">


## 1. Detection

YOLOv8 _detection_ models have no suffix and are the default YOLOv8 models, i.e. `yolov8n.pt` and are pretrained on COCO. See [Detection Docs](https://docs.ultralytics.com/tasks/detect/) for full details.


In [ ]:
# Load YOLOv8n, train it on COCO128 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model
model.train(data='coco128.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 2. Segmentation

YOLOv8 _segmentation_ models use the `-seg` suffix, i.e. `yolov8n-seg.pt` and are pretrained on COCO. See [Segmentation Docs](https://docs.ultralytics.com/tasks/segment/) for full details.


In [ ]:
# Load YOLOv8n-seg, train it on COCO128-seg for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')  # load a pretrained YOLOv8n segmentation model
model.train(data='coco128-seg.yaml', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

## 3. Classification

YOLOv8 _classification_ models use the `-cls` suffix, i.e. `yolov8n-cls.pt` and are pretrained on ImageNet. See [Classification Docs](https://docs.ultralytics.com/tasks/classify/) for full details.


In [ ]:
# Load YOLOv8n-cls, train it on mnist160 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')  # load a pretrained YOLOv8n classification model
model.train(data='mnist160', epochs=3)  # train the model
model('https://ultralytics.com/images/bus.jpg')  # predict on an image

# Appendix

Additional content below.

In [ ]:
# Git clone install (for development)
!git clone https://github.com/ultralytics/ultralytics -b main
%pip install -qe ultralytics

In [ ]:
# Run YOLOv8 tests (git clone install only)
!pytest ultralytics/tests

In [ ]:
# Validate multiple models
for x in 'nsmlx':
  !yolo val model=yolov8{x}.pt data=coco.yaml